In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import pickle
from app.util.timer import Timer
from app.util.Differ import Differ
from main import YoloRuntimeTest
from functools import partial

CPU input

In [7]:
args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml", 
    "type": "image",
    "show": False, 
    "score_threshold": 0.1, 
    "conf_threshold": 0.2, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "score_threshold": 0.1, 
    "conf_threshold": 0.2, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_1.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "score_threshold": 0.1, 
    "conf_threshold": 0.2, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

Initilize YOLO runtime test class

In [3]:
yolo_runtime_test = YoloRuntimeTest()

In [8]:
cpu_pytorch_image = yolo_runtime_test.ultralytics_run_image(args_pytorch)
print(cpu_pytorch_image)

[INFO] Initialize Model
[INFO] Inference Image


infer time: 0.13304789999995137 s
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_1.jpg: 480x640 9 cars, 5 traffic lights, 132.9ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_1.jpg: 480x640 9 cars, 5 traffic lights, 132.9ms
Speed: 2.0ms preprocess, 132.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Speed: 2.0ms preprocess, 132.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
Elapsed time: 0.1329 seconds
Class: car, Confidence: 0.85, Box: [970, 1035, 1066, 1119]
Class: car, Confidence: 0.84, Box: [803, 1049, 897, 1116]
Class: traffic light, Confidence: 0.84, Box: [514, 463, 565, 588]
Class: traffic light, Confidence: 0.84, Box: [36, 444, 102, 578]
Class: car, Confidence: 0.81, Box: [424, 1015, 708, 1217]
Class: car, Confidence: 0.71, Box: [381, 1064, 441, 1114]
Class: car, Confidence: 0.68, Box: [923, 1049, 958

In [ ]:
cpu_openvino_image = yolo_runtime_test.ultralytics_run_image(args_openvino)
print(cpu_openvino_image)

In [9]:
cpu_onnx_image = yolo_runtime_test.ultralytics_run_image(args_onnx)
print(cpu_onnx_image)

[INFO] Initialize Model
[INFO] Inference Image
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...
Loading app\weights\yolov9c.onnx for ONNX Runtime inference...


infer time: 0.12225300000000061 s
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_1.jpg: 640x640 9 cars, 5 traffic lights, 129.0ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_1.jpg: 640x640 9 cars, 5 traffic lights, 129.0ms
Speed: 1.5ms preprocess, 129.0ms inference, 14.5ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.5ms preprocess, 129.0ms inference, 14.5ms postprocess per image at shape (1, 3, 640, 640)
Elapsed time: 0.1290 seconds
Class: car, Confidence: 0.85, Box: [970, 1035, 1066, 1119]
Class: traffic light, Confidence: 0.84, Box: [514, 463, 565, 588]
Class: traffic light, Confidence: 0.84, Box: [36, 442, 102, 578]
Class: car, Confidence: 0.84, Box: [803, 1049, 897, 1116]
Class: car, Confidence: 0.80, Box: [42

In [ ]:
cpu_onnx_runtime_image = yolo_runtime_test.onnxruntime_run_image(args_onnx)
print(cpu_onnx_runtime_image)

Difference CPU

In [ ]:
pd.set_option('display.expand_frame_repr', False)

def generate_difference_df(image1, image2, label):
    differ = Differ(np.array(image1), np.array(image2))
    result = differ.find_difference()
    return pd.DataFrame(result, columns=[label, "cpu conf_diff", "cpu box_diff (px)"])

df_pt_openvino = generate_difference_df(cpu_pytorch_image[0], cpu_openvino_image[0], "pt vs openvino+ultralytics")
df_pt_onnx = generate_difference_df(cpu_pytorch_image[0], cpu_onnx_image[0], "pt vs onnx+ultralytics")
df_pt_onnxruntime = generate_difference_df(cpu_pytorch_image[0], cpu_onnx_runtime_image[0], "pt vs onnxruntime")

df_combined = pd.concat([df_pt_openvino, df_pt_onnx, df_pt_onnxruntime], axis=1)

print(df_combined)

Average CPU Time (10)

In [4]:
result_time = []

In [5]:
def run_inference_with_args(inference_func, args):
    def wrapper():
        return inference_func(args)
    return wrapper

def collect_execution_times(run_inference_func, args, iterations=10):
    execution_times = []
    for i in range(iterations):
        args["source"] = f"./app/assets/sample_image_{i}.jpg"
        wrapper_func = run_inference_with_args(run_inference_func, args)
        execution_time = wrapper_func()
        execution_times.append(execution_time[1] * 100)
    return execution_times

args_pytorch = {
    "weights": "./app/weights/yolov9c.pt", 
    "source": "./app/assets/sample_image_0.jpg",
    "classes": "./app/weights/metadata.yaml", 
    "type": "image",
    "show": False, 
    "score_threshold": 0.1, 
    "conf_threshold": 0.2, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_onnx = {
    "weights": "./app/weights/yolov9c.onnx", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "score_threshold": 0.1, 
    "conf_threshold": 0.2, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

args_openvino = {
    "weights": "./app/weights/yolov9c_openvino_model", 
    "source": "./app/assets/sample_image_0.jpg", 
    "classes": "./app/weights/metadata.yaml",
    "type": "image", 
    "show": False, 
    "score_threshold": 0.1, 
    "conf_threshold": 0.2, 
    "iou_threshold": 0.6, 
    "device": "cpu"
}

result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_pytorch))
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_openvino))
result_time.append(collect_execution_times(yolo_runtime_test.ultralytics_run_image, args_onnx))
result_time.append(collect_execution_times(yolo_runtime_test.onnxruntime_run_image, args_onnx))


[INFO] Initialize Model
[INFO] Inference Image


image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_0.jpg: 448x640 1 person, 1 bicycle, 6 cars, 3 trucks, 146.9ms
image 1/1 c:\Users\User\OneDrive\Desktop\pomper\Internship\yolov9\app\assets\sample_image_0.jpg: 448x640 1 person, 1 bicycle, 6 cars, 3 trucks, 146.9ms
Speed: 2.0ms preprocess, 146.9ms inference, 495.7ms postprocess per image at shape (1, 3, 448, 640)
Speed: 2.0ms preprocess, 146.9ms inference, 495.7ms postprocess per image at shape (1, 3, 448, 640)
Elapsed time: 0.1469 seconds
Class: car, Confidence: 0.93, Box: [558, 206, 808, 359]
Class: car, Confidence: 0.92, Box: [286, 210, 458, 352]
Class: car, Confidence: 0.91, Box: [465, 217, 596, 339]
Class: person, Confidence: 0.87, Box: [159, 143, 301, 403]
Class: truck, Confidence: 0.87, Box: [103, 90, 255, 316]
Class: truck, Confidence: 0.78, Box: [722, 170, 871, 346]
Class: truck, Confidence: 0.75, Box: [0, 154, 94, 354]
Class: bicycle, Confi

In [6]:
result_time = np.array(result_time)
df = pd.DataFrame(np.transpose(result_time), 
                  columns=["pytorch time cpu (ms)",
                           "openvino+ultralytics time cpu (ms)",
                           "onnx​+ultralytics time cpu (ms)", 
                           "onnx runtime time cpu (ms)"])
df.describe(percentiles=[.9, .95])

,pytorch time cpu (ms),openvino+ultralytics time cpu (ms),onnx​+ultralytics time cpu (ms),onnx runtime time cpu (ms)
count,10.000000,10.000000,10.000000,10.000000
mean,12.351315,12.286983,14.374077,16.701485
std,0.967538,0.239441,1.927969,2.097054
min,11.183453,12.054777,12.133741,12.947540
50%,12.110066,12.229943,13.717365,17.368840
90%,13.303833,12.481265,16.658485,18.666055
95%,13.997581,12.669148,17.437989,18.684933
max,14.691329,12.857032,18.217492,18.703810


Save CPU result

In [ ]:
with open('./app/saved_pkl/cpu_df.pkl', 'wb') as f:
    pickle.dump(df, f)